In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
from sklearn.cross_validation import train_test_split
import itertools
import nltk
from nltk.corpus import stopwords
import unicodedata
import sys
%matplotlib inline 

/Users/Lucy/anaconda/envs/capstone/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mypath = '/Users/Lucy/Google Drive/MSDS/2016Fall/DSGA1006/Data'
f = open(mypath + '/rnn-encoder-decoder/lstm_data.pickle','rb')
train, test = pickle.load(f)
f.close()

In [3]:
train.head()

,company_uuid,company_name,short_description,description,founded_on,clean_description,clean_short_description
4635,a1f100c4-49ce-cb02-625d-f3ec55c91e8e,ProGlove,A smart glove that helps workers in logistics ...,ProGlove is a professional wearable tool for p...,2014-12-30,ProGlove is a professional wearable tool for p...,A smart glove that helps workers in logistics ...
118053,ce9524dc-fb48-9e64-5a63-09ac6d5aedb2,Passlogix,Passlogix provides fast and secure access to e...,"Passlogix, Inc., specializes in providing fast...",1996-01-01,"Passlogix, Inc., specializes in providing fast...",Passlogix provides fast and secure access to e...
13505,ed0dacca-57c4-c1cf-00e0-0a3804b88219,SunFarmer,We're on a mission to provide reliable solar e...,SunFarmer is a nonprofit that provides the cap...,2013-03-01,SunFarmer is a nonprofit that provides the cap...,We're on a mission to provide reliable solar e...
76275,61834199-5997-0ee6-1a04-a339ecd13d40,Haus Bioceuticals,Haus Bioceuticals is a specialty pharmaceutica...,Haus Bioceuticals is a specialty pharmaceutica...,2009-01-01,Haus Bioceuticals is a specialty pharmaceutica...,Haus Bioceuticals is a specialty pharmaceutica...
118811,c56a2d51-a1f8-d797-56b1-fc953b7b185b,Kivera,Kivera foresaw the emergence of the wireless l...,"In 1997, the founders of Kivera foresaw the em...",1997-01-01,"In 1997, the founders of Kivera foresaw the em...",Kivera foresaw the emergence of the wireless l...


In [13]:
train_desc = train.clean_short_description.copy()
test_desc = test.clean_short_description.copy()
train_y = train.clean_short_description.copy()
test_y = test.clean_short_description.copy()

In [14]:
train_description = train_desc + train_y

In [15]:
train_y.head()

4635      A smart glove that helps workers in logistics ...
118053    Passlogix provides fast and secure access to e...
13505     We're on a mission to provide reliable solar e...
76275     Haus Bioceuticals is a specialty pharmaceutica...
118811    Kivera foresaw the emergence of the wireless l...
Name: clean_short_description, dtype: object

In [16]:
vocabulary_size = 5000
unknown_token = "UNKNOWNTOKEN"
sentence_start_token = "SENTENCESTART"
sentence_end_token = "SENTENCEEND"

In [17]:
# Split full descriptions into sentences
sentences = itertools.chain(*[[nltk.sent_tokenize(x.decode('utf-8').lower())] for x in train_description])
mod_sentences = []
for desc in sentences:
    full_desc = ""
    for x in desc:
        s = "%s %s %s " % (sentence_start_token, x, sentence_end_token)
        full_desc = full_desc + s
    mod_sentences.append(full_desc)

In [18]:
mod_sentences[:2]

[u'SENTENCESTART a smart glove that helps workers in logistics and manufacturing work more efficiently.a smart glove that helps workers in logistics and manufacturing work more efficiently. SENTENCEEND ',
 u'SENTENCESTART passlogix provides fast and secure access to enterprise resources with their flagship product; the v-go access accelerator suite.passlogix provides fast and secure access to enterprise resources with their flagship product; the v-go access accelerator suite. SENTENCEEND ']

In [19]:
def strip_punctuation(text):
    """
    >>> strip_punctuation(u'something')
    u'something'

    >>> strip_punctuation(u'something.,:else really')
    u'somethingelse really'
    """
    punctutation_cats = set(['Pc', 'Pd', 'Ps', 'Pe', 'Pi', 'Pf', 'Po'])
    return ''.join(x for x in text
                   if unicodedata.category(x) not in punctutation_cats)

In [20]:
mod_sentences = map(strip_punctuation, mod_sentences)

In [21]:
for i in mod_sentences[:5]:
    print i + '\n'

SENTENCESTART a smart glove that helps workers in logistics and manufacturing work more efficientlya smart glove that helps workers in logistics and manufacturing work more efficiently SENTENCEEND 

SENTENCESTART passlogix provides fast and secure access to enterprise resources with their flagship product the vgo access accelerator suitepasslogix provides fast and secure access to enterprise resources with their flagship product the vgo access accelerator suite SENTENCEEND 

SENTENCESTART were on a mission to provide reliable solar energy for schools hospitals and water projects around the worldwere on a mission to provide reliable solar energy for schools hospitals and water projects around the world SENTENCEEND 

SENTENCESTART haus bioceuticals is a specialty pharmaceutical company developing clinically active botanical extractshaus bioceuticals is a specialty pharmaceutical company developing clinically active botanical extracts SENTENCEEND 

SENTENCESTART kivera foresaw the emergen

In [22]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in mod_sentences]

In [23]:
for i in tokenized_sentences[3]:
    print i

SENTENCESTART
haus
bioceuticals
is
a
specialty
pharmaceutical
company
developing
clinically
active
botanical
extractshaus
bioceuticals
is
a
specialty
pharmaceutical
company
developing
clinically
active
botanical
extracts
SENTENCEEND


In [25]:
train_y[3]

"Lydia's iPhone and Android apps enable you to pay anyone very easily and for free."

In [26]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

Found 109713 unique words tokens.


In [27]:
#word_freq = sorted(word_freq.items(), key=lambda i: i[1], reverse=True)

In [28]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    


Using vocabulary size 5000.
The least frequent word in our vocabulary is 'hold' and appeared 16 times.


In [29]:
{word_to_index.keys()[word_to_index.values().index(k)]: k for k in sorted(word_to_index.values())[:2]}

{u'SENTENCEEND': 1, u'and': 0}

In [32]:
np.array(tokenized_sentences).shape

(50691,)